<a href="https://colab.research.google.com/github/adhi-thirumala/Science-Fair2022/blob/main/onlyindicdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install collections
!pip install datasets
!pip install transformers[sentencepiece]
!pip install tensorboard
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, AutoTokenizer, default_data_collator
import collections
import os
import sys
from google.colab import drive
drive.mount('/content/drive')
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ["WANDB_DISABLED"] = 'True'
 

BASE_PATH = "/content/drive/MyDrive/"


ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
Mounted at /content/drive


In [3]:
os.getcwd()

'/content'

In [4]:
from transformers import AutoModel, AutoTokenizer

hindi_tokenizer = AutoTokenizer.from_pretrained(BASE_PATH + "roberta-large-wechsel-hindi")
hindi_model = AutoModelForQuestionAnswering.from_pretrained(BASE_PATH + "roberta-large-wechsel-hindi")

tamil_tokenizer = AutoTokenizer.from_pretrained(BASE_PATH + "roberta-large-wechsel-tamil")
tamil_model = AutoModelForQuestionAnswering.from_pretrained(BASE_PATH + "roberta-large-wechsel-tamil")


Some weights of the model checkpoint at /content/drive/MyDrive/roberta-large-wechsel-hindi were not used when initializing RobertaForQuestionAnswering: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at /content/drive/MyDrive/roberta-large-wechsel-hindi and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN t

In [5]:
df_train_base = pd.read_csv(BASE_PATH + "train.csv")
df_test = pd.read_csv(BASE_PATH + "test.csv")
df_sub = pd.read_csv(BASE_PATH + "sample_submission.csv")